In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")
from keras.wrappers.scikit_learn import KerasClassifier
import keras.utils
from keras.utils import np_utils
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

In [8]:
df = pd.read_csv('train_age_dataset.csv')

df_ = pd.read_csv("test_age_dataset.csv") 

In [9]:
df.head()

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,...,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
0,265153,48958844,2,1,0.0,0.0,0.0,0,0.0,0.0,...,0.200000,0.0,0.041667,0.025000,0.0,0.000000,0.175000,0.033333,0.0,1
1,405231,51100441,2,2,0.0,0.0,0.0,0,0.0,0.0,...,0.093220,0.0,0.012712,0.018644,0.0,0.084746,0.000000,0.033898,82.5,2
2,57867,6887426,2,1,0.0,0.0,0.0,0,0.0,0.0,...,0.002786,0.0,0.000000,0.000557,0.0,0.002786,0.000000,0.000000,0.0,1
3,272618,50742404,2,1,0.0,0.0,0.0,0,0.0,0.0,...,0.008403,0.0,0.000000,0.001681,0.0,0.000000,0.000000,0.008403,0.0,1
4,251123,45589200,2,2,0.0,0.0,0.0,0,0.0,0.0,...,0.204918,0.0,0.000000,0.049180,0.0,0.008197,0.057377,0.180328,0.0,1


In [11]:
df.shape,df_.shape

((488877, 27), (54320, 26))

In [12]:
X_train = df.drop(columns=['age_group','Unnamed: 0'],axis=1)
y_train = df['age_group']
X_test = df_.drop(columns=['Unnamed: 0'],axis=1)

In [13]:
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
userId,488877.0,4.236096e+07,1.821664e+07,27.000000,3.537599e+07,4.336270e+07,5.370523e+07,7.904203e+07
tier,488877.0,1.975382e+00,4.307072e-01,1.000000,2.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00
gender,488877.0,1.213037e+00,4.094542e-01,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
following_rate,488877.0,8.223307e-02,2.467781e+00,0.000000,0.000000e+00,0.000000e+00,8.620690e-03,8.953040e+02
followers_avg_age,488877.0,3.475198e-01,8.163341e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00
following_avg_age,488877.0,4.030489e-01,9.252911e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00
max_repetitive_punc,488877.0,7.397484e-01,3.075954e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,6.240000e+02
num_of_hashtags_per_action,488877.0,2.771944e-04,1.222129e-02,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.333333e+00
emoji_count_per_action,488877.0,9.813795e-04,2.038139e-02,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00
punctuations_per_action,488877.0,1.280507e-02,1.595532e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.733333e+01


In [14]:
X_test.describe().T

,count,mean,std,min,25%,50%,75%,max
userId,54320.0,4.248125e+07,1.818136e+07,35.000000,3.547567e+07,4.344565e+07,5.370713e+07,7.904176e+07
tier,54320.0,1.976105e+00,4.311416e-01,1.000000,2.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00
gender,54320.0,1.211856e+00,4.086269e-01,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
following_rate,54320.0,8.120404e-02,3.103205e+00,0.000000,0.000000e+00,0.000000e+00,8.620690e-03,6.852778e+02
followers_avg_age,54320.0,3.440326e-01,8.130815e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00
following_avg_age,54320.0,3.974323e-01,9.179879e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00
max_repetitive_punc,54320.0,7.347938e-01,2.822803e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.300000e+02
num_of_hashtags_per_action,54320.0,3.572283e-04,1.552948e-02,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.333333e+00
emoji_count_per_action,54320.0,9.088367e-04,1.745622e-02,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.692308e+00
punctuations_per_action,54320.0,1.211719e-02,1.305760e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.250000e+01


In [18]:
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)
#test = sc.transform(df_)

In [19]:
#from xgboost import XGBClassifier
#classifier = XGBClassifier()
#classifier.fit(X_train, y_train)

In [20]:
#from sklearn.metrics import confusion_matrix, accuracy_score , classification_report
#y_pred = classifier.predict(X_test)
#cm = confusion_matrix(y_test, y_pred)
#print(cm) 
#accuracy_score(y_test, y_pred)

In [21]:
"""

loss=['hinge','log','modifier_huber','squared_hinge','perceptron']
penalty = ['li','l2','elasticnet']
alpha = [0.0001, 0.001,0.01,0.1,1,10,100,1000]
learnin_rate = ['constant','optimal','invscaling','adaptive']
class_weight = [{0.3,0.5,0.2},{0.3,0.4,0.3}]
eta0 = [1,10,100]

xg_class = XGBClassifier(objective = "multi:softmax", colsample_bytree = 1,
gamma = 1,subsample = 0.8, learning_rate = 0.01, max_depth = 3,
alpha = 10,n_estimators = 1000, multilabel_ =True, num_classes = 3)

#from sklearn.metrics import jaccard_score
from sklearn.model_selection import RandomizedSearchCV


param_distributions = dict(loss = loss, penalty=penalty, alpha=alpha, learnin_rate=learning_rate, class_weight=class_weight, eta0=eta0)
random = RandomizedSearchCV(estimator = xg_class, param_distributions=param_distributions, 
verbose = 1, n_jobs =-1, n_iter = 1000)

random_result = random.fit(X_train, y_train)

"""

'\n\nloss=[\'hinge\',\'log\',\'modifier_huber\',\'squared_hinge\',\'perceptron\']\npenalty = [\'li\',\'l2\',\'elasticnet\']\nalpha = [0.0001, 0.001,0.01,0.1,1,10,100,1000]\nlearnin_rate = [\'constant\',\'optimal\',\'invscaling\',\'adaptive\']\nclass_weight = [{0.3,0.5,0.2},{0.3,0.4,0.3}]\neta0 = [1,10,100]\n\nxg_class = XGBClassifier(objective = "multi:softmax", colsample_bytree = 1,\ngamma = 1,subsample = 0.8, learning_rate = 0.01, max_depth = 3,\nalpha = 10,n_estimators = 1000, multilabel_ =True, num_classes = 3)\n\n#from sklearn.metrics import jaccard_score\nfrom sklearn.model_selection import RandomizedSearchCV\n\n\nparam_distributions = dict(loss = loss, penalty=penalty, alpha=alpha, learnin_rate=learning_rate, class_weight=class_weight, eta0=eta0)\nrandom = RandomizedSearchCV(estimator = xg_class, param_distributions=param_distributions, \nverbose = 1, n_jobs =-1, n_iter = 1000)\n\nrandom_result = random.fit(X_train, y_train)\n\n'

In [ ]:
from xgboost import XGBClassifier
classifier = XGBClassifier(max_depth = 15,
                     gamma = 5,
                     subsample=0.9,
                     colsample_bytree=0.9,
                     n_estimators = 3000,
                     learning_rate = 0.19,
                     verbosity=2,
                     min_child_weight=1,
                     objective='multi:softmax',
                     nthread=4,
                     max_delta_step=1,
)
classifier.fit(X_train,y_train)

[03:00:29] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 442 extra nodes, 1218 pruned nodes, max_depth=15
[03:00:32] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 718 extra nodes, 1938 pruned nodes, max_depth=15
[03:00:34] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 766 extra nodes, 1844 pruned nodes, max_depth=15
[03:00:36] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 1438 extra nodes, 4684 pruned nodes, max_depth=15
[03:00:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore th

[03:02:18] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 434 extra nodes, 2406 pruned nodes, max_depth=15
[03:02:19] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 338 extra nodes, 2446 pruned nodes, max_depth=15
[03:02:22] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 538 extra nodes, 4140 pruned nodes, max_depth=15
[03:02:24] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 238 extra nodes, 1836 pruned nodes, max_depth=15
[03:02:26] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 338 extra nodes, 2604 pruned nodes, max_depth=15
[03:02:28] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree prunin

[03:03:53] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 154 extra nodes, 1068 pruned nodes, max_depth=15
[03:03:55] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 286 extra nodes, 1484 pruned nodes, max_depth=15
[03:03:57] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 304 extra nodes, 2546 pruned nodes, max_depth=15
[03:03:59] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 440 extra nodes, 2964 pruned nodes, max_depth=15
[03:04:02] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 298 extra nodes, 2228 pruned nodes, max_depth=15
[03:04:04] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree prunin

[03:05:36] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 214 extra nodes, 1874 pruned nodes, max_depth=15
[03:05:39] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 258 extra nodes, 4352 pruned nodes, max_depth=15
[03:05:41] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 146 extra nodes, 984 pruned nodes, max_depth=15
[03:05:44] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 218 extra nodes, 2450 pruned nodes, max_depth=15
[03:05:46] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 256 extra nodes, 2784 pruned nodes, max_depth=15
[03:05:48] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning

[03:07:20] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 64 extra nodes, 658 pruned nodes, max_depth=15
[03:07:22] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 106 extra nodes, 458 pruned nodes, max_depth=15
[03:07:24] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 164 extra nodes, 1486 pruned nodes, max_depth=15
[03:07:26] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 68 extra nodes, 862 pruned nodes, max_depth=15
[03:07:28] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 64 extra nodes, 632 pruned nodes, max_depth=15
[03:07:30] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 

...

[03:49:25] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 1150 pruned nodes, max_depth=0
[03:49:26] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 86 extra nodes, 1268 pruned nodes, max_depth=15
[03:49:28] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 38 extra nodes, 2044 pruned nodes, max_depth=9
[03:49:29] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 478 pruned nodes, max_depth=0
[03:49:30] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 84 extra nodes, 334 pruned nodes, max_depth=15
[03:49:31] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 38 ex

[03:51:03] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 16 extra nodes, 970 pruned nodes, max_depth=8
[03:51:05] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 2142 pruned nodes, max_depth=15
[03:51:06] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 58 extra nodes, 1300 pruned nodes, max_depth=14
[03:51:07] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 46 extra nodes, 790 pruned nodes, max_depth=15
[03:51:09] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 56 extra nodes, 1058 pruned nodes, max_depth=15
[03:51:10] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 0

[03:52:31] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 160 pruned nodes, max_depth=0
[03:52:33] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 226 pruned nodes, max_depth=0
[03:52:35] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 672 pruned nodes, max_depth=0
[03:52:37] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 36 extra nodes, 1658 pruned nodes, max_depth=13
[03:52:39] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 1540 pruned nodes, max_depth=15
[03:52:41] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 0 extra

KeyboardInterrupt: 

[03:53:03] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 68 extra nodes, 3352 pruned nodes, max_depth=15
[03:53:05] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 32 extra nodes, 480 pruned nodes, max_depth=14
[03:53:07] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 374 pruned nodes, max_depth=15
[03:53:09] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 406 pruned nodes, max_depth=0
[03:53:11] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 1228 pruned nodes, max_depth=14
[03:53:13] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_prune.cc:101: tree pruning end, 22 

In [ ]:
y_pred_test = classifier.predict(X_test)

In [ ]:
print((y_pred_test.reshape(len(y_pred_test),1)))

In [ ]:
ab = pd.DataFrame(y_pred_test.reshape(len(y_pred_test),1))

In [ ]:
ab.to_csv("sample_test13.csv", header = ["prediction"] , index = False)